**FEEL FREE TO UPVOTE**  （＾ｖ＾）

In [ ]:
#使用我的特征，队友的思路识别异常点，然后进行手动修改

In [ ]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
import time
warnings.filterwarnings('ignore')
np.random.seed(4950)

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_hist_trans = pd.read_csv('../input/historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('../input/new_merchant_transactions.csv')

In [ ]:
df_hist_trans = reduce_mem_usage(df_hist_trans)
df_new_merchant_trans = reduce_mem_usage(df_new_merchant_trans)

In [ ]:
for df in [df_hist_trans,df_new_merchant_trans]:
#     df['category_2'].fillna(1.0,inplace=True)
#     df['category_3'].fillna('A',inplace=True)
#     df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    #修改特征
    df['category_2'].fillna(-1,inplace=True)
    df['category_3'].fillna('other',inplace=True)
    df['merchant_id'].fillna('other',inplace=True)
#     df.loc[df['installments'].isin([999,-1]),'installments'] = 0 

In [ ]:
def get_new_columns(name,aggs):
    #for for 写法 nice
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [ ]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    df['month_diff'] = ((datetime.datetime(2018,6,1) - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']

In [ ]:
#因为根据auth_flag 将特征分成了两部分，这里聚合一个全局的auth_flag 特征
aggs = {}
aggs['purchase_amount'] = ['sum']
# aggs['installments'] = ['sum','max','min','mean','var','median']
aggs['authorized_flag'] = ['sum', 'mean','std']
aggs['card_id'] = ['size']
auth_flag = df_hist_trans.groupby(['card_id']).agg(aggs)
auth_flag.columns = get_new_columns('auth_flag',aggs)
auth_flag.reset_index(inplace=True)
df_train = df_train.merge(auth_flag,on='card_id',how='left')
df_test = df_test.merge(auth_flag,on='card_id',how='left')
del auth_flag
gc.collect()

In [ ]:
#对 authorized_flag进行结果编码
aggs = {}
for col in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
    df_hist_trans[col+'_auth_mean'] = df_hist_trans.groupby([col])['authorized_flag'].transform('mean')
    df_hist_trans[col+'_auth_sum'] = df_hist_trans.groupby([col])['authorized_flag'].transform('sum') 
    aggs[col+'_auth_mean'] = ['mean']
    aggs[col+'_auth_sum'] = ['sum'] 
auth_encoder = df_hist_trans.groupby(['card_id']).agg(aggs)
auth_encoder.columns = get_new_columns('auth_encoder',aggs)
auth_encoder.reset_index(inplace=True)
df_train = df_train.merge(auth_encoder,on='card_id',how='left')
df_test = df_test.merge(auth_encoder,on='card_id',how='left')
del auth_encoder
gc.collect()

In [ ]:
def aggregate_per_month(prefix,history,agg_func):
    grouped = history.groupby(['card_id', 'month_lag'])
    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = [prefix + '_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = [prefix + '_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True) 
    return final_group

In [ ]:
#对授权码 进行按月聚合
agg_func = {'authorized_flag': [ 'sum', 'mean','median']}
final_group =  aggregate_per_month('agg_per_month_total',df_hist_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group

In [ ]:
authorized_transactions = df_hist_trans[df_hist_trans['authorized_flag'] == 1]
df_hist_trans = df_hist_trans[df_hist_trans['authorized_flag'] == 0]

In [ ]:
#使用countVectorizer对category特征进行处理，别人号称可以提升 千分之3
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(token_pattern='\w{1,}')   

In [ ]:
df_hist_trans.shape

In [ ]:
agg_func = {'purchase_amount': [ 'sum', 'mean', 'min', 'max', 'std']}
final_group =  aggregate_per_month('agg_per_month_auth',authorized_transactions,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
final_group =  aggregate_per_month('agg_per_month_hist',df_hist_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
final_group =  aggregate_per_month('agg_per_month_hist_new',df_new_merchant_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
gc.collect()

In [ ]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

In [ ]:
i = 0
for df in [authorized_transactions,df_hist_trans]:
    aggs = {}
    for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',
                'state_id','city_id']:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum','max','min','mean','var','median']
    aggs['installments'] = ['sum','max','min','mean','var','median']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var','median']
    aggs['month_diff'] = ['mean','median']
#     aggs['authorized_flag'] = ['sum', 'mean','median']
    aggs['weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']
    #产生交叉特征，内存有问题
    features = ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id'
               ,'merchant_id']
#     for coli in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#         for colj in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#             df[coli + colj] = df[coli].astype(str) + df[colj].astype(str)
#             features.append(coli + colj)
    for col in features:
        df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
        df[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum') 
        aggs[col+'_mean'] = ['mean']
        aggs[col+'_sum'] = ['sum'] 
#         添加特征，使用outlier进行编码
#         outliers_mean = df.groupby([col])['outliers'].mean()
#         outliers_sum = df.groupby([col])['outliers'].sum()
#         df[col+'_outliers_mean'] = df[col].map(outliers_mean)
#         df[col+'_outliers_sum'] = df[col].map(outliers_sum)
#         aggs[col+'_outliers_mean'] = ['mean']
#         aggs[col+'_outliers_sum'] =['sum']   
        
    if i == 0:
        prefix = 'auth_hist'
    else:
        prefix = 'hist'
    new_columns = get_new_columns(prefix,aggs)
    i += 1
    # df_hist_trans.sort_values(['card_id','purchase_date'],inplace = True)

    df_hist_trans_group = df.groupby('card_id').agg(aggs)
    df_hist_trans_group.columns = new_columns
    df_hist_trans_group.reset_index(drop=False,inplace=True)
    df_hist_trans_group[prefix + '_purchase_date_diff'] = (df_hist_trans_group[prefix + '_purchase_date_max'] - df_hist_trans_group[prefix + '_purchase_date_min']).dt.days
    df_hist_trans_group[prefix + '_purchase_date_average'] = df_hist_trans_group[prefix + '_purchase_date_diff']/df_hist_trans_group[prefix + '_card_id_size']
    df_hist_trans_group[prefix + '_purchase_date_uptonow'] = (datetime.datetime(2018,6,1) - df_hist_trans_group[prefix + '_purchase_date_max']).dt.days
    #下面这个特征考虑了：有的人可能购买频率较低，但是还是忠实粉丝的情况
#     df_hist_trans_group[prefix + '_purchase_date_uptonow_ave'] =  df_hist_trans_group[prefix + '_purchase_date_uptonow']/df_hist_trans_group[prefix + '_purchase_date_average']

    #每一个card中未授权消费次数
#     df_hist_trans_group[prefix + '_unauthorized_number'] = df_hist_trans_group[prefix + '_card_id_size'] - df_hist_trans_group[prefix + '_authorized_flag_sum']
    #最近活跃时间，确实是一个强特征，感觉可以再挖出来几个特征，比如最近5次消费时间，最近10次消费时间，如果值比较小，说明最近很活跃
    #没有效果
    # grouped =  df_hist_trans.groupby('card_id')['purchase_date']
    # df_hist_trans_group['hist_purchase_5thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(5)).dt.days
    # df_hist_trans_group['hist_purchase_3thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(3)).dt.days
    # df_hist_trans_group['hist_purchase_10thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(10)).dt.days
    df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
    df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
    del df_hist_trans_group
    time.sleep(5)
    gc.collect()
    del df;gc.collect()
    time.sleep(5)

In [ ]:
aggs = {}
#添加特征
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',
            'state_id','city_id']:
    aggs[col] = ['nunique']
    
aggs['purchase_amount'] = ['sum','max','min','mean','var','median']
aggs['installments'] = ['sum','max','min','mean','var','median']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var','median']
aggs['month_diff'] = ['mean','median']
aggs['weekend'] = ['sum', 'mean']
aggs['authorized_flag'] = ['sum', 'mean','median']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

#添加特征   
features = ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id'
               ,'merchant_id']
#产生交叉特征，内存有问题
# for coli in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#     for colj in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#         df_new_merchant_trans[coli + colj] = df_new_merchant_trans[coli].astype(str) + df_new_merchant_trans[colj].astype(str)
#         features.append(coli + colj)
for col in features:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    df_new_merchant_trans[col+'_sum'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('sum')
    aggs[col+'_mean'] = ['mean']
    aggs[col+'_sum'] = ['sum']
    #添加特征，使用outlier进行编码
#     outliers_mean = df.groupby([col])['outliers'].mean()
#     outliers_sum = df.groupby([col])['outliers'].sum()
#     df[col+'_outliers_mean'] = df[col].map(outliers_mean)
#     df[col+'_outliers_sum'] = df[col].map(outliers_sum)
#     aggs[col+'_outliers_mean'] = ['mean']
#     aggs[col+'_outliers_sum'] =['sum']   
    
new_columns = get_new_columns('new_hist',aggs)
# df_new_merchant_trans.sort_values(['card_id','purchase_date'],inplace = True)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.datetime(2018,6,1) - df_hist_trans_group['new_hist_purchase_date_max']).dt.days
#下面这个特征考虑了：有的人可能购买频率较低，但是还是忠实粉丝的情况
# df_hist_trans_group['new_hist_purchase_date_uptonow_ave'] =  df_hist_trans_group['new_hist_purchase_date_uptonow']/df_hist_trans_group['new_hist_purchase_date_average']

#每一个card中未授权消费次数
df_hist_trans_group['new_hist_unauthorized_number'] = df_hist_trans_group['new_hist_card_id_size'] - df_hist_trans_group['new_hist_authorized_flag_sum']
# grouped = df_new_merchant_trans.groupby('card_id')['purchase_date']
# df_hist_trans_group['new_hist_purchase_5thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(5)).dt.days
# df_hist_trans_group['new_hist_purchase_3thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(3)).dt.days
# df_hist_trans_group['new_hist_purchase_10thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(10)).dt.days

df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group
gc.collect()
del df_new_merchant_trans
gc.collect()
time.sleep(5)

In [ ]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime(2018,6,1) - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    #添加特征
    df['auth_hist_first_buy'] = (df['auth_hist_purchase_date_min'] - df['first_active_month']).dt.days
    #修改特征
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min','auth_hist_purchase_date_max','auth_hist_purchase_date_min']:
#         del df[f]
        df[f] = df[f].astype(np.int64) * 1e-9
    for f in ['auth_encoder_category_2_auth_sum_sum','auth_encoder_category_3_auth_sum_sum',
            'auth_encoder_state_id_auth_sum_sum','auth_encoder_subsector_id_auth_sum_sum',
            'auth_encoder_merchant_category_id_auth_sum_sum','auth_encoder_city_id_auth_sum_sum']:
        df[f] = df[f].astype(np.int64) * 1e-9
#         del df[f]
    #上面auth_flag已经聚合过了card_id_size ,purchase_amount
#     df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']  + df['auth_hist_card_id_size']
#     df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']+df['auth_hist_purchase_amount_sum']
#添加特征

for f in ['feature_1','feature_2','feature_3','month','dayofweek']:
    order_label1 = df_train.groupby([f])['outliers'].mean()
    df_train[f+'_outliers_mean'] = df_train[f].map(order_label1)
    df_test[f+'_outliers_mean'] = df_test[f].map(order_label1)
    
    order_label2 = df_train.groupby([f])['outliers'].sum()
    df_train[f+'_outliers_sum'] = df_train[f].map(order_label2)
    df_test[f+'_outliers_sum'] = df_test[f].map(order_label2)
    
#     order_label1 = df_train.groupby([f])['target'].mean()
#     df_train[f+'_target_mean'] = df_train[f].map(order_label1)
#     df_test[f+'_target_sum'] = df_test[f].map(order_label1)
#     order_label2 = df_train.gorupby([f])['target'].sum()
#     df_train[f+'_target_sum'] = df_train[f].map(order_label2)
#     df_test[f+'_target_sum'] = df_test[f].map(order_label2)
 
# get_dummies 似乎有一点点不良影响
df_train = pd.get_dummies(df_train,columns =['feature_1','feature_2'])
df_test = pd.get_dummies(df_test,columns =['feature_1','feature_2'])

In [ ]:
#首次购买的时间居然早于首次激活的时间，进行调整
df_train.loc[df_train['auth_hist_first_buy'] < 0, 'auth_hist_first_buy'] = -1
df_train.loc[df_train['hist_first_buy'] < 0, 'hist_first_buy'] = -1
df_train.head()

In [ ]:
exclude_features = []
exclude_features += ['card_id', 'first_active_month','target','outliers']
df_train_columns = [c for c in df_train.columns if c not in exclude_features]
target = df_train['target']
del df_train['target']

In [ ]:
len(df_train_columns)
len(exclude_features)

In [ ]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 42,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 48,
         "scale_pos_weight": 15,
         "random_state": 4950}
# folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=4950)
# oof = np.zeros(len(df_train))
# predictions = np.zeros(len(df_test))

# for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
#     print("fold {}".format(fold_))
#     trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=df_train['outliers'].iloc[trn_idx])#, categorical_feature=categorical_feats)
#     val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=df_train['outliers'].iloc[val_idx])#, categorical_feature=categorical_feats)

#     num_round = 10000
#     clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
#     oof[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
    
#     predictions += clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration) / folds.n_splits

# from sklearn.metrics import roc_auc_score
# oof_label = [1 if c > 0.5 else 0 for c in oof]
# roc_auc_score(oof_label, df_train['outliers'])

In [ ]:
#使用下面的方法进行交叉验证确定最佳 boosting round 500
trn_data = lgb.Dataset(df_train[df_train_columns], label=df_train['outliers'].values)#, categorical_feature=categorical_feats)
lgb_cv = lgb.cv(param, trn_data, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

In [ ]:
clf = lgb.train(param, trn_data, 500, verbose_eval=100, valid_sets=(trn_data))
te_prob = clf.predict(df_test[df_train_columns])

In [ ]:
sub_df = pd.DataFrame({"card_id":df_test["card_id"].values})
sub_df["target"] = te_prob
sub_df.to_csv("outlier_prob.csv", index=False)